In [53]:
import numpy as np
import pandas as pd
from numpy import log2 as log
eps = np.finfo(float).eps
t_data = pd.read_csv("./dataset/train.csv")

In [54]:
t_data = t_data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Fare'], axis = 1)
t_data = t_data.dropna()

In [55]:
t_data['Age'] = t_data['Age'].interpolate() #fixing missing values of age

In [56]:
#Entropy of the entire dataset
entropy = 0
values = t_data['Survived'].unique()
for value in values:
    p = t_data['Survived'].value_counts()[value]/len(t_data.Survived)
    entropy = entropy + (-(p*log(p)))
print(entropy)

0.9735190023846809


In [57]:
def attribute_entropy(col):
    print("Calculating att-ent")
    y = t_data['Survived'].unique()
    x = t_data[col].unique()
    entropy = 0
    for i in x:
        e = 0
        for j in y:
            num = len(t_data[col][t_data[col] == i][t_data['Survived'] == j])
            den = len(t_data[col][t_data[col] == i])
            p = num/(den+eps)
            e = e + (-(p*log(p+eps)))
        p2 = den/len(t_data)
        entropy = entropy + (-(p2*e))
    return abs(entropy)

In [58]:
print(attribute_entropy('Age'))

Calculating att-ent
0.846271645340514


In [59]:
def find_max(d):
    print("finding max")
    data = d.drop(['Survived'], axis = 1)
    Entropy_att = []
    IG = []
    #print(data.keys())
    for key in data.keys()[:-1]:
        IG.append(entropy-attribute_entropy(key))
    #print(IG)
    return data.keys()[np.argmax(IG)]
find_max(t_data)

finding max
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent


'Sex'

In [62]:
def get_subtable(df, node,value):
    print('making a subtable')
    return df[df[node] == value].reset_index(drop=True)


def buildTree(data): 
    
    node = find_max(data)
    
    vals = np.unique(data[node])
    #print(vals)
    tree = {}
    tree[node] = {}
    for val in vals:
        subtable = get_subtable(data, node, val)
        #print((subtable.head()))
        
        if(len(set(subtable['Survived'])) == 1):
            tree[node][val] = unique[0]
        else:
            tree[node][val] = buildTree(subtable)
    return tree

tree = buildTree(t_data)
print(tree)
  

finding max
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
making a subtable
finding max
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
making a subtable
finding max
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
making a subtable
finding max
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
making a subtable
finding max
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
making a subtable
finding max
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
making a subtable
finding max
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-ent
making a subtable
finding max
Calculating att-ent
Calculating att-ent
Calculating att-ent
Calculating att-en

KeyboardInterrupt: 